<a href="https://colab.research.google.com/github/KarimWalid805/MSDS680-ML-COLAB/blob/main/CourseProject_MSDS680_KarimWalid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Machine Learning (MSDS680) Course Project**
This projects aims to find the best performing model that can train on the EMNIST letters data.


---


*   Author: Karim Walid
*   Date: 30/03/2025
*   Instructor: Pantelis Kaplanoglou


---



# **Option 01**

In [ ]:
# Install the Rapid Deep Neural Networks library
!pip install radnn -q
!pip list | grep "radnn"

from radnn.system.hosts import ColabHost

# Mount to the project folder that is under the lesson folder
ColabHost().detect_workspace(["CS345", "MSDS680"]).change_to_project_dir("MLData")

In [ ]:
import os
os.environ["TF_USE_LEGACY_KERAS"]="1"   # Use Keras 2.x. We set this before the first import of tensorflow

import csv
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from radnn import mlsys, FileSystem
from radnn.experiment import MLExperimentConfig
#from mllib.visualization import CPlot, CPlotTrainingLogs, CAutoMultiImagePlot
oFileSys = FileSystem(config_folder="MLConfig", model_folder="MLModels", dataset_folder="MLData")
# __________ | Settings | __________
IS_PLOTING_DATA             = True
IS_RETRAINING               = False
RANDOM_SEED             = 2022
mlsys.filesys = FileSystem("MLConfig", model_folder="MLModels", dataset_folder="MLData")

In [ ]:
import numpy as np
import gzip
os.getcwd()

In [ ]:
import numpy as np
from enum import Enum


# =========================================================================================================================
class SetType(Enum):
  TRAINING_SET      = 1
  VALIDATION_SET    = 2
  UNKNOWN_TEST_SET  = 3
# =========================================================================================================================


# =========================================================================================================================
class CDataSet(object):
  # --------------------------------------------------------------------------------------
  # Constructor
  def __init__(self):
    # ................................................................
    # // Fields \\
    self.Samples            = None
    self.Labels             = None
    self.SampleCount        = 0
    self.FeatureCount       = None
    self.ClassCount         = None

    self.TSSamples      = None
    self.TSLabels       = None
    self.TSSampleCount  = 0

    self.VSSamples      = None
    self.VSLabels       = None
    self.VSSampleCount  = 0

    self.USSamples      = None
    self.USLabels       = None
    self.USSampleCount  = 0
    # ................................................................
  # --------------------------------------------------------------------------------------
  # Method
  def DebugPrint(self):
    print("Shape of sample tensor", self.Samples.shape)
    print('.'*80)

    print("Datatype of sample tensor before convertion: %s" % str(self.Samples.dtype))
    # Convert the data to 32bit floating point numbers (default for faster computations)
    self.Samples = np.asarray(self.Samples, dtype=np.float32)
    print("Datatype of sample tensor after convertion: %s" % str(self.Samples.dtype))
    print('.'*80)

    # Classification into 2 classes == Binary classification
    print("Class labels")
    print(self.Labels)
    print('.'*80)
  # --------------------------------------------------------------------------------------
  def LoadSet(self, p_nSamples, p_nLabels, p_nType):
    if p_nType == SetType.TRAINING_SET:
      self.TSSamples = p_nSamples
      self.TSLabels  = p_nLabels
      self.TSSampleCount = self.TSSamples.shape[0]
      if self.FeatureCount is None:
        self.FeatureCount = self.TSSamples.shape[1]
        self.ClassCount = len(np.unique(self.TSLabels))
    elif p_nType == SetType.VALIDATION_SET:
      self.VSSamples = p_nSamples
      self.VSLabels = p_nLabels
      self.VSSampleCount = self.VSSamples.shape[0]

    self.SampleCount = self.TSSampleCount + self.VSSampleCount + self.USSampleCount
# =========================================================================================================================

In [ ]:
import os
import numpy as np
import pandas as pd
import gzip


class EMNISTDataSet(CDataSet):
  # --------------------------------------------------------------------------------------
  # Constructor
  def __init__(self, filestore):
    super(EMNISTDataSet, self).__init__()

    with gzip.open(filestore.file("emnist-letters-test-images-idx3-ubyte.gz"), 'rb') as f:
        dTestImages = np.frombuffer(f.read(), np.uint8, offset = 16).reshape(-1, 28 ,28, 1)

    with gzip.open(filestore.file("emnist-letters-train-images-idx3-ubyte.gz"), 'rb') as f:
        dTrainImages = np.frombuffer(f.read(), np.uint8, offset = 16).reshape(-1, 28 ,28, 1)

    with gzip.open(filestore.file("emnist-letters-test-labels-idx1-ubyte.gz"), 'rb') as f:
        dTestLabels = np.frombuffer(f.read(), np.uint8, offset = 8)

    with gzip.open(filestore.file("emnist-letters-train-labels-idx1-ubyte.gz"), 'rb') as f:
        dTrainLabels = np.frombuffer(f.read(), np.uint8, offset = 8)


    self.LoadSet(dTrainImages, dTrainLabels, SetType.TRAINING_SET)
    self.LoadSet(dTestImages, dTestLabels, SetType.VALIDATION_SET)
  # --------------------------------------------------------------------------------------



In [ ]:
# _____// Data Hyperparameters \\_____
oDatasetFS = oFileSys.datasets.subfs("EMNIST")


oEMNIST  = EMNISTDataSet(oDatasetFS)
sDataName  = "EMNIST"
nSamples   = oEMNIST.TSSamples
nLabels    = oEMNIST.TSLabels

vSamples = oEMNIST.VSSamples
vLabels    = oEMNIST.VSLabels

print("Loaded %s dataset" % sDataName)
print("Training set shape:", nSamples.shape)
print("Class count:", len(np.unique(nLabels)))

In [ ]:
oDatasetFS.obj.save(nSamples, "nSamples.pkl")
oDatasetFS.obj.save(nLabels, "nLabels.pkl")
oDatasetFS.obj.save(vSamples, "vSamples.pkl")
oDatasetFS.obj.save(vLabels, "vLabels.pkl")

# **Option 02**

The three options of traning CNN01(worst),CNN02(better) and CNN03(best)

In [ ]:
CONFIG_CNN_01 = MLExperimentConfig(number=1).assign({
                 "ModelName": "EMNIST_CNN"
                ,"CNN.InputShape": [28,28,1]
                ,"CNN.Classes": 26
                ,"CNN.ModuleCount": 6
                ,"CNN.ConvOutputFeatures": [9,16,24,32,48,48]
                ,"CNN.ConvWindows": [ [3,2,True], [3,1,True] ,  [3,1,True], [3,2,True], [3,1,True], [3,1,True] ]
                ,"CNN.PoolWindows": [  None      , None       ,  None      , None      , [3,2]     , None      ]
                ,"CNN.HasBatchNormalization": True
                ,"Training.MaxEpoch": 12
                ,"Training.BatchSize": 500
                ,"Training.LearningRate": 0.001
                ,"Experiment.RandomSeed": 2022
            })

oConfig = CONFIG_CNN_01

In [ ]:
CONFIG_CNN_02 = MLExperimentConfig(number=2).assign({
     "ModelName": "Better_MNIST_CNN"
    ,"CNN.InputShape": [28,28,1]
    ,"CNN.Classes": 26
    ,"CNN.ModuleCount": 8  # Increased depth
    ,"CNN.ConvOutputFeatures": [16, 32, 48, 64, 128, 128, 256, 256]  # More feature maps
    ,"CNN.ConvWindows": [ [3,2,True], [3,1,True], [5,1,True], [3,2,True], [3,1,True], [3,1,True], [3,1,True], [3,2,True] ]
    ,"CNN.PoolWindows": [  [2,2] , None , [2,2], None , None , [2,2], None , None]  # More pooling
    ,"CNN.HasBatchNormalization": True
    ,"CNN.HasDropout": True  # Adding dropout for regularization
    ,"CNN.DropoutRate": 0.3  # Dropout rate at 30%
    ,"Training.MaxEpoch": 20  # More epochs for better convergence
    ,"Training.BatchSize": 256  # Lower batch size for better generalization
    ,"Training.LearningRate": 0.001
    ,"Training.LearningRateDecay": 0.95  # Decay learning rate over epochs
    ,"Experiment.RandomSeed": 2022
})

oConfig = CONFIG_CNN_02

In [ ]:
CONFIG_CNN_03 = MLExperimentConfig(number=3).assign({
     "ModelName": "Best_MNIST_CNN"
    ,"CNN.InputShape": [28,28,1]
    ,"CNN.Classes": 26
    ,"CNN.ModuleCount": 8  # Increased depth
    ,"CNN.ConvOutputFeatures": [16, 32, 48, 64, 128, 128, 256, 256, 512, 512]  # less feature maps
    ,"CNN.ConvWindows": [ [3,2,True], [3,1,True], [5,1,True], [3,2,True], [3,1,True], [3,1,True], [3,1,True], [3,2,True] ]
    ,"CNN.PoolWindows": [  [2,2] , None , [2,2], None , None , [2,2], None , None]  # More pooling
    ,"CNN.HasBatchNormalization": True
    ,"CNN.HasDropout": True  # Adding dropout for regularization
    ,"CNN.DropoutRate": 0.3  # Dropout rate at 30%
    ,"Training.MaxEpoch": 20  # More epochs for better convergence
    ,"Training.BatchSize": 128 # Lower batch size for better generalization
    ,"Training.LearningRate": 0.0001
    ,"Training.LearningRateDecay": 0.90  # Decay learning rate over epochs
    ,"Experiment.RandomSeed": 2022
})

oConfig = CONFIG_CNN_03

In [ ]:
import numpy as np
import tensorflow as tf
from radnn.data import DataSetBase


class MNISTDataFeed(object):

  # --------------------------------------------------------------------------------------
  def __init__(self, p_oDataSet, p_oConfig, p_nFeatureCount=None):
    super(MNISTDataFeed, self).__init__()
    # ................................................................

    # // Fields \\
    self.DataSet: CDataSet  = p_oDataSet
    self.FileStore          = oFileSys.datasets.subfs("EMNIST")
    self.PredictBatchSize = None
    if "Prediction.BatchSize" in p_oConfig:
      self.PredictBatchSize   = p_oConfig["Prediction.BatchSize"]
    self.TrainingBatchSize  = p_oConfig["Training.BatchSize"]

    # Calculate mean and std
    oDataSetStats = self.FileStore.obj.load("EMNIST-meanstd.pkl")
    if oDataSetStats is not None:
      self.PixelMean    = oDataSetStats["mean"]
      self.PixelStd     = oDataSetStats["std"]
    else:
      self.calculateAndSaveDatasetStats()

    if "ClassCount" in p_oConfig:
      self.ClassCount = p_oConfig["ClassCount"]
    else:
      self.ClassCount = p_oConfig["CNN.Classes"]
    if "InputShape" in p_oConfig:
      self.InputShape = p_oConfig["InputShape"]
    else:
      self.InputShape = p_oConfig["CNN.InputShape"]
    self.PaddingOffset = 3
    self.PaddingTarget = self.InputShape[0] + 3


    self.TSFeed       = self.CreateTrainingDataFeed((  self.DataSet.TSSamples
                                                      ,self.DataSet.TSLabels), self.TrainingBatchSize)
    self.TSRecallFeed = None
    self.VSFeed       = self.CreateValidationDataFeed((self.DataSet.VSSamples,
                                                        self.DataSet.VSLabels), 100)
    # ................................................................
  # --------------------------------------------------------------------------------------
  def calculateAndSaveDatasetStats(self):
    self.PixelMean = np.mean(self.DataSet.TSSamples, axis=(0,1,2))
    self.PixelStd  = np.std(self.DataSet.TSSamples, axis=(0,1,2))
    oDataSetStats = { "mean": self.PixelMean, "std": self.PixelStd}
    self.FileStore.obj.save(oDataSetStats,"EMNIST-meanstd.pkl")
  # --------------------------------------------------------------------------------------------------------
  def denormalize_image(self, normed_image):
      image = (normed_image * self.PixelStd) + self.PixelMean
      image = image.astype(np.uint8)
      return image
  # --------------------------------------------------------------------------------------------------------
  def normalizeImage(self, p_nImage):
      normed_image = (p_nImage - self.PixelMean) / self.PixelStd
      return normed_image
  # --------------------------------------------------------------------------------------------------------
  def randomCrop(self, image):
      # CIFAR: We follow the simple data augmentation in [24] for training: 4 pixels are padded on each side,
      # and a  32×32  crop is  randomly  sampled  from  the  paddedimage or its horizontal flip.

      #distorted_image = image
      distorted_image = tf.image.pad_to_bounding_box(image, self.PaddingOffset, self.PaddingOffset
                                                          ,  self.PaddingTarget,  self.PaddingTarget)    # pad 3 pixels to each side
      distorted_image = tf.image.random_crop(distorted_image, self.InputShape)
      # [WARNING] Flip is a non-label preserving transformation for ExShapes
      # distorted_image = tf.image.random_flip_left_right(distorted_image)
      return distorted_image
  # -----------------------------------------------------------------------------------
  def PreprocessTrainingImageAugmentDataset(self, p_tImageInTS, p_tLabelInTS):
    # Original feed that was used in experiments did not contain the following line but the bug was not triggered (cached image format float32?)
    tImage = tf.cast(p_tImageInTS, tf.float32)  # //[BF] overflow of standardization
    tNormalizedImage = self.normalizeImage(tImage)

    tNewRandomImage  = self.randomCrop(tNormalizedImage)

    tTargetOneHot = tf.one_hot(p_tLabelInTS, self.ClassCount)

    return tNewRandomImage, tTargetOneHot
  # --------------------------------------------------------------------------------------
  def CreateTrainingDataFeed(self, p_oDataTuple, p_nBatchSize):
    oTSData = tf.data.Dataset.from_tensor_slices(p_oDataTuple)
    oTSData = oTSData.map(self.PreprocessTrainingImageAugmentDataset, num_parallel_calls=8)
    #oTSData = oTSData.cache() # This reduced accuracy on the cifar10 data feed
    oTSData = oTSData.shuffle(self.DataSet.TSSampleCount)
    oTSData = oTSData.batch(p_nBatchSize)
    #oTSData = oTSData.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    print("Training data feed object:", oTSData)
    return oTSData
  # -----------------------------------------------------------------------------------
  def PreprocessValidationImageWithID(self, p_tImageID, p_tImageInVS, p_tLabelInVS):
    tImage = tf.cast(p_tImageInVS, tf.float32)  # //[BF] overflow of standardization
    tNormalizedImage = self.normalizeImage(tImage)

    tTargetOneHot = tf.one_hot(p_tLabelInVS, self.ClassCount)

    return (p_tImageID, tNormalizedImage), tTargetOneHot
  # -----------------------------------------------------------------------------------
  def PreprocessValidationImage(self, p_tImageInVS, p_tLabelInVS):
    tImage = tf.cast(p_tImageInVS, tf.float32)  # //[BF] overflow of standardization
    tNormalizedImage = self.normalizeImage(tImage)

    tTargetOneHot = tf.one_hot(p_tLabelInVS, self.ClassCount)

    return tNormalizedImage, tTargetOneHot
  # -----------------------------------------------------------------------------------
  def CreateValidationDataFeed(self, p_oDataTuple, p_nBatchSize=None):
    nArgsCount = len(list(p_oDataTuple))
    if nArgsCount == 2:
      oData = tf.data.Dataset.from_tensor_slices(p_oDataTuple)
      oData = oData.map(self.PreprocessValidationImage, num_parallel_calls=8)
    elif nArgsCount == 3:
      oData = tf.data.Dataset.from_tensor_slices(p_oDataTuple)
      oData = oData.map(self.PreprocessValidationImageWithID, num_parallel_calls=8)

    if p_nBatchSize is None:
      p_nBatchSize = self.DataSet.vs_sample_count
    oData = oData.batch(p_nBatchSize)
    print("Validation data feed object:", oData)
    return oData
  # --------------------------------------------------------------------------------------

In [ ]:
oData = MNISTDataFeed(oEMNIST, oConfig)

print("Training data feed object:", oData.TSFeed)
print("Validation data feed object:", oData.VSFeed)

In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import InputLayer, Flatten, Dense, BatchNormalization, Activation, Softmax
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.regularizers import L2
# =========================================================================================================================
class CCNNCustom(keras.Model):
  # --------------------------------------------------------------------------------------
  # Constructor
  def __init__(self, p_oConfig):
    super(CCNNCustom, self).__init__()

    # ..................... Object Attributes ...........................
    self.Config = p_oConfig

    self.InputShape         = self.Config["CNN.InputShape"]
    self.ClassCount         = self.Config["CNN.Classes"]
    self.ModuleCount        = self.Config["CNN.ModuleCount"]

    self.ConvLayerFeatures  = self.Config["CNN.ConvOutputFeatures"]
    self.ConvWindows        = self.Config["CNN.ConvWindows"]
    self.PoolWindows        = self.Config["CNN.PoolWindows"]

    if "CNN.HasBatchNormalization" not in self.Config:
        self.Config["CNN.HasBatchNormalization"] = False

    self.KerasLayers        = []

    self.OutputLayer        = None
    self.SoftmaxActivation  = None
    self.Input              = None
    self.Structure          = None
    # ...................................................................

    # Default values for extra customization

    if "CNN.ActivationFunction" not in self.Config:
        self.Config["CNN.ActivationFunction"] = "relu"

    if "CNN.ConvHasBias" not in self.Config:
        self.Config["CNN.ConvHasBias"] = False

    if "CNN.KernelInitializer" not in self.Config:
        self.Config["CNN.KernelInitializer"] = "glorot_uniform"

    if "CNN.BiasInitializer" not in self.Config:
        self.Config["CNN.BiasInitializer"] = "zeros"

    if "Training.RegularizeL2" not in self.Config:
        self.Config["Training.RegularizeL2"] = False

    if "Training.WeightDecay" not in self.Config:
        self.Config["Training.WeightDecay"] =  1e-5

    if self.Config["Training.RegularizeL2"]:
        print("Using L2 regularization of weights with weight decay %.6f" % self.Config["Training.WeightDecay"])


    self.Create()
  # --------------------------------------------------------------------------------------
  def Create(self):                # override a virtual in our base class
    # This loop creates stacked convolutional modules of the form   CONVOLUTION - ACTIVATION - NORMALIZATION - MAX POOLING
    for nModuleIndex in range(0, self.ModuleCount):
      nFeatures     = self.ConvLayerFeatures[nModuleIndex]
      oConvWindowSetup = self.ConvWindows[nModuleIndex]
      nWindowSize   = oConvWindowSetup[0]
      nStride       = oConvWindowSetup[1]

      sPaddingType      = "valid"
      if len(oConvWindowSetup) == 3:
          bIsPadding    = oConvWindowSetup[2]
          if bIsPadding:
              sPaddingType = "same"

      if self.Config["Training.RegularizeL2"]:
          oWeightRegularizer = L2(self.Config["Training.WeightDecay"])
      else:
          oWeightRegularizer = None

      oConvolution = Conv2D(nFeatures, kernel_size=nWindowSize, strides=nStride, padding=sPaddingType
                            , use_bias=self.Config["CNN.ConvHasBias"]
                            , kernel_regularizer=oWeightRegularizer
                            , kernel_initializer=self.Config["CNN.KernelInitializer"]
                            , bias_initializer=self.Config["CNN.BiasInitializer"])
      self.KerasLayers.append(oConvolution)

      oActivation  = Activation(self.Config["CNN.ActivationFunction"])
      self.KerasLayers.append(oActivation)

      if self.Config["CNN.HasBatchNormalization"]:
          oNormalization = BatchNormalization()
          self.KerasLayers.append(oNormalization)

      oPoolWindow   = self.PoolWindows[nModuleIndex]
      # Set the pool size to None for a module that does not do Max Pooling.
      if oPoolWindow is not None:
          nPoolSize   = oPoolWindow[0]
          nPoolStride = oPoolWindow[1]
          oMaxPooling = MaxPooling2D(pool_size=[nPoolSize, nPoolSize], strides=[nPoolStride, nPoolStride])
          self.KerasLayers.append(oMaxPooling)


    # After the stack of convolutional modules, the activation cube will be flattened to a vector using a Flatten keras layer
    self.FlatteningLayer = Flatten()


    # The output layer for the classifier is a fully connected (dense) that has one neuron for each class.
    # You might consider the stack of convolutional modules functioning as the "hidden" layer in the 2-layer NN architecture.
    if self.Config["Training.RegularizeL2"]:
        oWeightRegularizer = L2(self.Config["Training.WeightDecay"])
    else:
        oWeightRegularizer = None
    self.OutputLayer = Dense(self.ClassCount, use_bias=True
                             ,kernel_regularizer=oWeightRegularizer )

    # Instead of using sigmoid for each neuron, we use the softmax activation function so that neuron "fire" together.
    self.SoftmaxActivation = Softmax()
  # --------------------------------------------------------------------------------------------------------
  def call(self, p_tInput):        # overrides a virtual in keras.Model class
    bPrint = self.Structure is None
    if bPrint:
        self.Structure = []

    self.Input = p_tInput

    # ....... Convolutional Feature Extraction  .......
    # Feed forward to the next layer
    tA = p_tInput
    for nIndex,oKerasLayer in enumerate(self.KerasLayers):
        if bPrint:
            self.Structure.append([nIndex + 1, str(tA.name), str(tA.shape)])
        tA = oKerasLayer(tA)

    # Flattens the activation cube to a vector
    tA = self.FlatteningLayer(tA)
    if bPrint:
        nIndex += 1
        self.Structure.append([nIndex + 1, str(tA.name), str(tA.shape)])

    # ....... Classifier  .......
    # Fully connected (dense) layer that has a count of neurons equal to the classes, with softmax activation function
    tA = self.OutputLayer(tA)
    if bPrint:
        nIndex += 1
        self.Structure.append([nIndex + 1, str(tA.name), str(tA.shape)])

    tA = self.SoftmaxActivation(tA)
    if bPrint:
        nIndex += 1
        self.Structure.append([nIndex + 1, str(tA.name), str(tA.shape)])


    return tA
  # --------------------------------------------------------------------------------------
# =========================================================================================================================

In [ ]:
oConfig = CONFIG_CNN_03 #change the model here - Options: ("CONFIG_CNN_01"/"CONFIG_CNN_02"/"CONFIG_CNN_03")
oCNN = CCNNCustom(oConfig)

# -----------------------------------------------------------------------------------
def LRSchedule(epoch, lr):
    if epoch == 10:
        nNewLR = lr * 0.5
        print("Setting LR to %.5f" % nNewLR)
        return nNewLR
    else:
        return lr
# -----------------------------------------------------------------------------------

nInitialLearningRate    = oConfig["Training.LearningRate"]

oCostFunction  = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
oOptimizer = tf.keras.optimizers.Adam(learning_rate=nInitialLearningRate)
oCallbacks = None

In [ ]:
print("-------------------- Experiment Configuration ------------------") #Check which one you chose
print(oConfig)

In [ ]:
from radnn import mlsys
mlsys.random_seed_all(oConfig["Experiment.RandomSeed"])
print(oConfig)

In [ ]:
IS_RETRAINING = True

oModelFS = oFileSys.models.subfs(oConfig.experiment_code)
sModelFolder = oModelFS.base_folder
# Append the .keras extension to the model folder path
sModelFolder = sModelFolder + ".keras"  # or sModelFolder + ".h5" for HDF5 format
sProcessLogFileName = "train.history"

if (not os.path.isdir(sModelFolder)) or IS_RETRAINING:
  oCNN.compile(loss=oCostFunction, optimizer=oOptimizer, metrics=["accuracy"])
  oProcessLog = oCNN.fit(  oData.TSFeed, batch_size=oConfig["Training.BatchSize"]
                            ,epochs=oConfig["Training.MaxEpoch"]
                            ,validation_data=oData.VSFeed
                          )
  print(f"Saving to {sModelFolder}")
  oCNN.save(sModelFolder)
  oHistory = oProcessLog.history
  oModelFS.obj.save(oHistory, sProcessLogFileName, is_overwriting=True)
else:
  print(f"Loading from {sModelFolder}")
  oCNN = tf.keras.models.load_model(sModelFolder)
  oHistory = oModelFS.obj.load(sProcessLogFileName)
oCNN.summary()

In [ ]:
import csv

sFileName = oModelFS.file(f"Model-Structure-{oConfig.experiment_code}.csv")

with open(sFileName, "w") as f:
    write = csv.writer(f)
    for oItem in oCNN.Structure:
        print(oItem)
        write.writerow(oItem)

# **Model Evaluation**

In [ ]:
from radnn.plots import PlotLearningCurve

sPlotFileName = oFileSys.models.file(f'{oConfig.experiment_code}-LearningCurve-%s.png')
oPlot = PlotLearningCurve(oHistory, oConfig.experiment_code)
oPlot.prepare().save(sPlotFileName % "Accuracy").show()
oPlot.prepare_cost("CCE").save(sPlotFileName % "CCE").show()

In [ ]:
nPredictedProbabilities = oCNN.predict(oData.VSFeed)
nPredictedClassLabels  = np.argmax(nPredictedProbabilities, axis=1)

nTargetClassLabels     = oEMNIST.VSLabels

for nIndex, nProbs in enumerate(nPredictedProbabilities):
  if nIndex < 10:
    print("#%.2d Predicted:%d (Probabilities:%s) Actual:%d" % (nIndex+1, nPredictedClassLabels[nIndex], nProbs, nTargetClassLabels[nIndex])) # [PYTHON] Format string example
    print("  |__ Sum of all output neuron activations:%.3f" % np.sum(nProbs))




In [ ]:
from radnn.evaluation import EvaluateClassification

# We create an evaluator object that will produce several metrics
oEvaluator = EvaluateClassification(nTargetClassLabels, nPredictedClassLabels)
oEvaluator.print_per_class()
oEvaluator.print_overall()
oEvaluator.print_confusion_matrix()